In [86]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage.filters import correlate 
import matplotlib.pyplot as plt
import matplotlib.image as pltimg 
from scipy.spatial.distance import cdist


In [59]:
def load_gray_to_double(path):
    '''
    :param path: path to load image
    :return: grayscaled and float image
    '''
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # transmit the image to float
    img = img.astype(np.float64)/255.0

    print(img.shape)
    return img

def load_rgb(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    return img


def show_img(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    
    
def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

In [60]:
"""
Harris Corner Detector
Usage: Call the function harris(filename) for corner detection
Reference   (Code adapted from):
             http://www.kaij.org/blog/?p=89
             Kai Jiang - Harris Corner Detector in Python
             
"""
from pylab import *
from scipy import signal
from scipy import *
import numpy as np
from PIL import Image

def harris(filename, min_distance = 10, threshold = 0.1):
    """
    filename: Path of image file
    threshold: (optional)Threshold for corner detection
    min_distance : (optional)Minimum number of pixels separating 
     corners and image boundary
    """
    im = np.array(Image.open(filename).convert("L"))
    harrisim = compute_harris_response(im)
    filtered_coords = get_harris_points(harrisim, min_distance, threshold)
    plot_harris_points(im, filtered_coords)
    return filtered_coords

def gauss_derivative_kernels(size, sizey=None):
    """ returns x and y derivatives of a 2D 
        gauss kernel array for convolutions """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    y, x = mgrid[-size:size+1, -sizey:sizey+1]
    #x and y derivatives of a 2D gaussian with standard dev half of size
    # (ignore scale factor)
    gx = - x * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    gy = - y * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    return gx,gy

def gauss_kernel(size, sizey = None):
    """ Returns a normalized 2D gauss kernel array for convolutions """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    x, y = mgrid[-size:size+1, -sizey:sizey+1]
    g = exp(-(x**2/float(size)+y**2/float(sizey)))
    return g / g.sum()

def compute_harris_response(im):
    """ compute the Harris corner detector response function 
        for each pixel in the image"""
    #derivatives
    gx,gy = gauss_derivative_kernels(3)
    imx = signal.convolve(im,gx, mode='same')
    imy = signal.convolve(im,gy, mode='same')
    #kernel for blurring
    gauss = gauss_kernel(3)
    #compute components of the structure tensor
    Wxx = signal.convolve(imx*imx,gauss, mode='same')
    Wxy = signal.convolve(imx*imy,gauss, mode='same')
    Wyy = signal.convolve(imy*imy,gauss, mode='same')   
    #determinant and trace
    Wdet = Wxx*Wyy - Wxy**2
    Wtr = Wxx + Wyy   
    return Wdet / Wtr

def get_harris_points(harrisim, min_distance=10, threshold=0.1):
    """ return corners from a Harris response image
        min_distance is the minimum nbr of pixels separating 
        corners and image boundary"""
    #find top corner candidates above a threshold
    corner_threshold = max(harrisim.ravel()) * threshold
    harrisim_t = (harrisim > corner_threshold) * 1    
    #get coordinates of candidates
    candidates = harrisim_t.nonzero()
    coords = [ (candidates[0][c],candidates[1][c]) for c in range(len(candidates[0]))]
    #...and their values
    candidate_values = [harrisim[c[0]][c[1]] for c in coords]    
    #sort candidates
    index = argsort(candidate_values)   
    #store allowed point locations in array
    allowed_locations = zeros(harrisim.shape)
    allowed_locations[min_distance:-min_distance,min_distance:-min_distance] = 1   
    #select the best points taking min_distance into account
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i][0]][coords[i][1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i][0]-min_distance):(coords[i][0]+min_distance),
                (coords[i][1]-min_distance):(coords[i][1]+min_distance)] = 0               
    return filtered_coords

def plot_harris_points(image, filtered_coords):
    """ plots corners found in image"""
    figure()
    gray()
    imshow(image)
    plot([p[1] for p in filtered_coords],[p[0] for p in filtered_coords],'r*')
    axis('off')
    show()

#harris('./CS543_ECE549 Assignment 3_files/sample_panorama.JPG')

In [83]:
"""
    Extract  descriptor

"""
def descriptor_extract(img, neighborhoods, feature_points):
    num_of_features_ponts = len(feature_points)
    print(num_of_features_ponts)
    descriptors = np.zeros((num_of_features_ponts, (2*neighborhoods+1)**2))
    tmp_img = img.copy()
    
    for i in range(num_of_features_ponts):
        # obtain img slices
        # !! Remember to +1 for slice right part! 
        tmp_img = img[feature_points[i][0]-neighborhoods:
                      feature_points[i][0]+neighborhoods+1,
                  feature_points[i][1]-neighborhoods:
                  feature_points[i][1]+neighborhoods+1]
        descriptors[i, :] = tmp_img.reshape((1, (2*neighborhoods+1)**2))
        
    return descriptors

In [ ]:
# RANSAC
# matches: number * 
# ((points_left_x,points_right_y,points_right_x,points_right_y))
def RANSAC(matches):
    num_Of_selected = len(matches)
    sample_numbers = 4
    threshold = 5
    iterations = 300
    
    for i in range(iterations):
        if sample_numbers == 4:
            # get sample from matches
            inliers = random.sample(num_Of_selected ,sample_numbers)
        A = []
        for j in range(sample_numbers):
            current_match = matches[inliers[j]]
            
    
function [inliers, num_of_inliers, mean_of_residual, H_re] = RANSAC(matches)

    % threshold
    threshold = 5;
    
    % num of iterations
    iterations = 250;

    % Use four matches to initialize the homography in each iteration. 
    num_of_samples = 4;
    n = 1;    
    num_of_matches = size(matches, 1);

    while(n < iterations)
        if num_of_samples == 4
            inliers = randsample(num_of_matches, num_of_samples);
        end
        A = [];
        for i = 1:num_of_samples
            current_match = matches(inliers(i), :);
            xT = [current_match(2), current_match(1), 1];
            A = [A; xT*0, xT, xT*(-current_match(3))];
            A = [A; xT, xT*0, xT*(-current_match(4))];
        end
        
        % Homography fitting calls for homogeneous least squares.
        [~, ~, V] = svd(A);
        H = V(:, end);
        H_re = reshape(H, 3, 3);
        
        num_of_inliers = 0;
        inliers = [];
        residual = [];
        for i = 1:num_of_matches
            X =  H_re' * [matches(i, 2); matches(i, 1); 1];
            x = X(1) / X(3);
            y = X(2) / X(3);
            if (dist2([x, y], [matches(i, 4), matches(i, 3)]) < threshold)
                inliers = [inliers; i];
                residual = [residual; dist2([x,y], [matches(i, 4), matches(i, 3)])];
                num_of_inliers = num_of_inliers + 1;
            end
        end

        if (num_of_inliers < 15)
            num_of_samples = 4;
        else
           num_of_samples = num_of_inliers;
           n = n + 1;
        end
    end

    mean_of_residual = mean(residual);
    num_of_inliers




In [75]:
def gauss_img(img, sigma):
    print(img.shape)
    tmp_img = img
    kernel = gauss_kernel(2*sigma+1)
    print(img.ndim)
    if img.ndim == 3:
        for i in range(img.ndim):
            tmp_img[:, :, i] = correlate(tmp_img[:, :, i], kernel)
    else:
        tmp_img = correlate(tmp_img, kernel)
    return tmp_img
        

In [76]:
img_1_left_path = "data\\part1\\left.jpg"
img_1_right_path = "data\\part1\\right.jpg"

In [81]:
# load image
img1_left_rgb = load_rgb(img_1_left_path)
img1_right_rgb = load_rgb(img_1_right_path)
img1_left_gray = load_gray_to_double(img_1_left_path)
img1_right_gray = load_gray_to_double(img_1_right_path)
# img1_left = load_image(img_1_left_path)
# img1_right = load_image(img_2_right_path)
# show_img(img1_left)
# show_img(img1_right)
neighborhoods = 5

# harris Cornor detect feature points 
feature_points_left = harris(img_1_left_path, min_distance=2*neighborhoods)
feature_points_right = harris(img_1_right_path, min_distance=2*neighborhoods)
# print(feature_points_left)




(398, 800)
(398, 800)


C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in true_divide


In [84]:
"""
Extract local neighborhoods around every keypoint in both images
and form descriptors simply by "flattening" the pixel values in each neighborhood 
to one-dimensional vectors.
"""
descriptor_left = descriptor_extract(img1_left_gray, 
                                      neighborhoods, feature_points_left)
descriptor_right = descriptor_extract(img1_right_gray, 
                                       neighborhoods, feature_points_right)


598
376


In [90]:
"""
Compute distances between every descriptor in one image and every descriptor in the other image. 
In Python, you can use scipy.spatial.distance.cdist(X,Y,'sqeuclidean') 
for fast computation of Euclidean distance. 
"""
# dim of distance: num_Of_left * num_of_right
distance = cdist(descriptor_left, descriptor_right, 'sqeuclidean')
"""
Select putative matches based on the matrix of pairwise descriptor distances obtained above. 
You can select all pairs whose descriptor distances are below a specified threshold, 
or select the top few hundred descriptor pairs with the smallest pairwise distances.
"""

# num of points to extract 
total_select_numbers = min(300, len(feature_points_right), len(feature_points_left))
select_match = []
INF = 11111
for i in range(total_select_numbers):
    # find matrix minimum index
    ri, ci = numpy.unravel_index(A.argmin(), A.shape)
    select_match += [(feature_points_left[ri][0],feature_points_left[ri][1],
                      feature_points_right[ci][0],feature_points_right[ci][1])]
    # set to INF after used
    distance[ri, :] = INF
    distance[:, ci] = INT
    
"""
Implement RANSAC to estimate a homography mapping one image onto the other. 
Report the number of inliers and the average residual for the inliers 
(squared distance between the point coordinates in one image and
 the transformed coordinates of the matching point in the other image). 
Also, display the locations of inlier matches in both images.
"""
 

    


(598, 376)
(598, 121)
(376, 121)


'\nSelect putative matches based on the matrix of pairwise descriptor distances obtained above. \nYou can select all pairs whose descriptor distances are below a specified threshold, \nor select the top few hundred descriptor pairs with the smallest pairwise distances.\n'